# SimPy Example for IE 306.02
This example shows how a simple G/G/1 system (1 server with an infinite capacity queue, random interarrival and service times) can be modeled with a process-interaction view using the SimPy library. 

In this example customers place calls to the call center of a company at randomn times. There is only a single operator in this call center, and picks up the first call waiting when she is available. The customers are assumed to be extremely paint, as they wait as long as it gets to talk to the operator.

In [84]:
import simpy
import random

Define a set of globals that define the characteristics of the model instance to be simulated. This includes the seed (RANDOM_SEED) for the random number generators, and key parameters for the interarrival (i.e. mean arrival rate) and service time (i.e. lower and upper bounds for the range) distribution.

In [85]:
Total_Time = 100000

RANDOM_SEED = 978
INTERARRIVAL_RATE = 0.1 
HEALING_TIME_RANGE = [50, 90]
random.seed(RANDOM_SEED)

hospital_treating_time_rate = 6
home_treating_time_rate = 10
r = 1 + random.random()
hospital2home_treating_time_rate = 6*r

num_of_individuals = 0
S = 147 + 177 + 207
if(S > 1000): num_of_individuals = S
elif(S > 10): num_of_individuals = S + 1000
else: num_of_individuals = S*300

num_of_beds = num_of_individuals/12
exponential_rate = num_of_individuals/300

num_of_patients = int(random.expovariate(num_of_individuals/Total_Time))
print(num_of_individuals, num_of_patients)

1531 10


Define the necessary set of arrays for bookkeeping

In [86]:
service_times = [] #Duration of the conversation between the customer and the operator (Service time)
queue_w_times = [] #Time spent by a customer while it waits for the operator (Queue waiting time Wq)

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to a uniform distribution.

In [87]:
class Patient(object):
    def __init__(self, name, env, opr):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.try_to_heal())
    
    
    def try_to_heal(self):
        if(self.is_going_hospital() == False):
            print('%s at home at %g' % (self.name, self.env.now))
            yield self.env.process(self.healing())
            print('%s is healed at %g in home' % (self.name, self.env.now))
        else:
            print('%s arrives hospital at %g' % (self.name, self.env.now))
            with operator.request() as req:
                yield req
                print('%s is assigned to an operator at %g' % (self.name, self.env.now))
                queue_w_times.append(self.env.now - self.arrival_t)
                yield self.env.process(self.healing())
                print('%s is healed at %g' % (self.name, self.env.now))
            
            
    def healing(self):
        if(self.is_going_hospital):
            duration = random.expovariate(hospital_treating_time_rate)
        elif(self.is_going_hospital):
            duration = random.expovariate(hospital2home_treating_time_rate)
        else:
            duration = random.expovariate(home_treating_time_rate)
        yield self.env.timeout(duration)
        service_times.append(duration)
        
        
    def is_going_hospital(self):
        random_number = random.random()
        if(random_number > 0.2): return False
        else: return True
    

In [88]:
def patient_generator(env, operator):
    """Generate new patients with using exponential rate!!!."""
    for i in range(num_of_patients):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        patient = Patient('Patient %s' %(i+1), env, operator)  

In [89]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = num_of_beds)
env.process(patient_generator(env, operator))
env.run()

Patient 1 at home at 0.975969
Patient 1 is healed at 1.07132 in home
Patient 2 at home at 17.1839
Patient 2 is healed at 17.3405 in home
Patient 3 arrives hospital at 33.6839
Patient 3 is assigned to an operator at 33.6839
Patient 3 is healed at 34.015
Patient 4 at home at 40.0728
Patient 4 is healed at 40.1238 in home
Patient 5 at home at 42.1504
Patient 5 is healed at 42.3678 in home
Patient 6 at home at 46.6703
Patient 6 is healed at 46.9853 in home
Patient 7 at home at 83.1811
Patient 7 is healed at 83.2244 in home
Patient 8 arrives hospital at 102.645
Patient 8 is assigned to an operator at 102.645
Patient 8 is healed at 102.862
Patient 9 at home at 110.02
Patient 9 is healed at 110.458 in home
Patient 10 arrives hospital at 115.118
Patient 10 is assigned to an operator at 115.118
Patient 10 is healed at 115.366


In [90]:
print(queue_w_times)

[0.0, 0.0, 0.0]


In [91]:
print(service_times)

[0.0953520620836633, 0.15658744335720862, 0.331093254575622, 0.05100799674898696, 0.217474233214506, 0.3149850023530053, 0.0432156806747217, 0.21699724412420188, 0.43823813222665886, 0.24792682854756953]
